# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.8 ダブルクロスバリデーション (Double Cross-Validation, DCV)
### サポートベクターマシン (Support Vector Machine, SVM)

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

SVM は 2 つのクラスを分類するための手法です。あやめのデータセットには、setosa, versicolor, virginica の 3 つのクラスがあるため、setosa と versicolor でまとめて 1 つのクラス `setosa+versicolor` とします。

In [ ]:
dataset.iloc[0:100,0] = 'setosa+versicolor'

In [ ]:
dataset # 念のため確認

In [ ]:
# y と x に分割
y = dataset.iloc[:,0]
x = dataset.iloc[:,1:]

In [ ]:
estimated_y_in_outer_cv = y.copy() # 外側の CV による y の推定結果を格納する変数

In [ ]:
estimated_y_in_outer_cv # 念のため確認。実際の y と同じものになっています

In [ ]:
outer_fold_number = 10 # 外側の CV における fold 数

In [ ]:
fold_number = 5 # 内側の CV における fold 数

外側の CV の分割

In [ ]:
indexes = [] # fold の番号を格納する変数

In [ ]:
for sample_number in range(x.shape[0]):
    indexes.append(sample_number % outer_fold_number)

In [ ]:
indexes # 念のため確認

参考1 : for 文で list の変数を作成するとき、以下のリスト内包表記を用いることでコードがシンプルになり、また実行時間が短縮されます

In [ ]:
indexes = [sample_number % outer_fold_number for sample_number in range(x.shape[0])]

参考2 : 以下のようにすると for 文を使わずに `indexes` を準備できます

In [ ]:
import numpy as np # NumPy のインポート
from numpy import matlib
min_number = x.shape[0] // outer_fold_number
mod_number = x.shape[0] % outer_fold_number
indexes = np.matlib.repmat(np.arange(outer_fold_number), 1, min_number).ravel()
if mod_number != 0:
    indexes = np.r_[indexes, np.arange(mod_number)]

`indexes` をシャッフル

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
np.random.seed(99) # 再現性のため乱数の種を固定
fold_index_in_outer_cv = np.random.permutation(indexes) # シャッフル
np.random.seed() # 乱数の種の固定を解除

In [ ]:
fold_index_in_outer_cv # 念のため確認

ガウシアンカーネルを用いた SVM で DCV

In [ ]:
nonlinear_svm_cs = 2 ** np.arange(-10, 11, 1.0) # Cの候補

In [ ]:
nonlinear_svm_cs # 念のため確認

In [ ]:
nonlinear_svm_gammas = 2 ** np.arange(-20, 11, 1.0) #ガウシアンカーネルのγの候補

In [ ]:
nonlinear_svm_gammas # 念のため確認

In [ ]:
from sklearn.model_selection import StratifiedKFold # 内側の CV の分割の設定に使用

In [ ]:
fold = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=9) # 内側の CV の分割の設定。(KFold ではなく) StratifiedKFold を使用することで、fold ごとのクラスの割合がなるべく同じになるように分割されます

In [ ]:
from sklearn.svm import SVC # SVM の実行に使用

In [ ]:
model_for_cross_validation = SVC(kernel='rbf')

内側の CV におけるグリッドサーチの設定

In [ ]:
from sklearn.model_selection import GridSearchCV # グリッドサーチに使用

In [ ]:
gs_cv = GridSearchCV(model_for_cross_validation, {'C':nonlinear_svm_cs, 'gamma':nonlinear_svm_gammas}, cv=fold) # クロスバリデーションによるグリッドサーチの設定

DCV の実行

In [ ]:
# 外側の CV
for fold_number_in_outer_cv in range(outer_fold_number):
    print(fold_number_in_outer_cv + 1, '/', outer_fold_number)
    # トレーニングデータとテストデータに分割
    x_train = x.iloc[fold_index_in_outer_cv != fold_number_in_outer_cv, :]
    y_train = y.iloc[fold_index_in_outer_cv != fold_number_in_outer_cv]
    x_test = x.iloc[fold_index_in_outer_cv == fold_number_in_outer_cv, :]
    # 特徴量の標準化 (オートスケーリング)
    autoscaled_x_train = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0,ddof=1)
    autoscaled_x_test = (x_test - x_train.mean(axis=0)) / x_train.std(axis=0,ddof=1)
    # 内側の CV におけるグリッドサーチの実行
    gs_cv.fit(autoscaled_x_train, y_train)
    optimal_nonlinear_svm_c = gs_cv.best_params_['C'] # 最適な C
    optimal_nonlinear_svm_gamma = gs_cv.best_params_['gamma'] # 最適な γ
    # トレーニングデータを用いたモデル構築
    model = SVC(kernel='rbf', C=optimal_nonlinear_svm_c, gamma=optimal_nonlinear_svm_gamma) # SVM モデルの宣言
    model.fit(autoscaled_x_train, y_train) # SVMモデル構築
    # テストデータの推定
    estimated_y_test = model.predict(autoscaled_x_test) # テストデータの推定
    estimated_y_in_outer_cv[fold_index_in_outer_cv==fold_number_in_outer_cv] = estimated_y_test # 推定結果を格納

In [ ]:
estimated_y_in_outer_cv # 念のため確認

DCV における混同行列、正解率

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_dcv = pd.DataFrame(metrics.confusion_matrix(y, estimated_y_in_outer_cv, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_dcv.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_dcv.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_dcv # 確認

In [ ]:
confusion_matrix_dcv.to_csv('confusion_matrix_dcv.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y, estimated_y_in_outer_cv) # 正解率

In [ ]:
estimated_y_in_outer_cv = pd.DataFrame(estimated_y_in_outer_cv) # pandas の DataFrame 型に変換

In [ ]:
estimated_y_in_outer_cv.columns = ['estimated_class']

In [ ]:
estimated_y_in_outer_cv.to_csv('estimated_y_dcv.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してくださいa

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。